# SIADS 516: Homework 3

- **Dr. Chris Teplovs**, School of Information, University of Michigan
- **Kris Steinhoff**, School of Information, University of Michigan


This homework assignment builds on the Spark DataFrame material we covered in class.

You will be using a compressed version of the Yelp Academic Dataset.  The data set is provided for you in the assets/data/yelp_academic of your workspace and you should not need to download it again if you're working on the Coursera hosted notebook environment.

You might want to refer to the lecture companion notebooks (in resources/lecture_notebooks/ or equivalently via Coursera as "Ungraded Lab: Spark Core Demo" and "Ungraded Lab: Spark SQL Demo) for hints about libraries to import, etc.

You will notice that there are a **lot** of reviews.  You might want to work off a small sample (i.e. use the sample() function in Spark) to work on a reduced size dataset while you're developing your solution.

In [1]:
# The AutograderHelper class provides methods used by the autograder.
from autograder_helper import AutograderHelper

In [2]:
# Autograder cell. This cell is worth 0 points.
# This cell has hidden code used to configure the autograder.

In [3]:
from pyspark.sql import SparkSession 

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName('My First Spark application') \
    .getOrCreate()
sc = spark.sparkContext

22/07/24 13:44:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
# Set up some RDDs:
user = spark.read.json('../../assets/data/yelp_academic/yelp_academic_dataset_user.json.gz')
review = spark.read.json('../../assets/data/yelp_academic/yelp_academic_dataset_review.json.gz')
checkin = spark.read.json('../../assets/data/yelp_academic/yelp_academic_dataset_checkin.json.gz')

In [5]:
from pyspark.sql.functions import explode, udf
from pyspark.sql.types import *
from pyspark.ml.feature import * 
import re
import string

In [6]:
%%time
import pyspark.sql.functions as f
from pyspark.sql.functions import udf, explode
from pyspark.sql.types import ArrayType,StringType
from pyspark.sql.functions import hour, col

CPU times: user 8 µs, sys: 5 µs, total: 13 µs
Wall time: 15.3 µs


In [7]:
review.columns

['business_id',
 'cool',
 'date',
 'funny',
 'review_id',
 'stars',
 'text',
 'useful',
 'user_id']

## -- COOL COMPLIMENTS --

Determine how many users have received more than 5000 "cool" compliments.

- Create a variable `user_count` (an integer) which contains the number of user with more than 5000 "cool" compliments (using the `compliment_cool` field.)

In [8]:
# YOUR CODE HERE
#raise NotImplementedError()
user1 = user.filter(user['compliment_cool'] > 5000).collect()
user_count = len(user1)

In [9]:
assert type(user_count) == int, "The user_count variable should be an integer."

In [10]:
# Autograder cell. This cell is worth 2 points (out of 20). This cell contains hidden tests.

## -- USEFUL POSITIVE REVIEWS --

Determine the top 5 most useful positive reviews.

- Create a variable `top_5_useful_positive`. This should be a PySpark DataFrame
- For this question a "positive review" is one with 4 or 5 stars
- The DataFrame should be ordered by `useful` and contain 5 rows
- The DataFrame should have these columns:
    - `review_id`
    - `useful`
    - `stars`

In [11]:
%%time
top_5_useful_positive = review['review_id','useful','stars'].filter(review['stars'] >= 4) \
                                                 .sort(col("useful").desc()).head(5)

top_5_useful_positive = spark.createDataFrame(top_5_useful_positive)

top_5_useful_positive = top_5_useful_positive.select(top_5_useful_positive.review_id, top_5_useful_positive.useful, top_5_useful_positive.stars)

top_5_useful_positive.toPandas()

CPU times: user 137 ms, sys: 68.9 ms, total: 206 ms
Wall time: 44.7 s


,review_id,useful,stars
0,1lGXlyq4MALOMx17vpBcoQ,358,5.0
1,gAUkgn4dTO-R2n5LBcht-w,278,5.0
2,5S985RjfmDJYsJvUt-3uJQ,244,4.0
3,0nr6SQFKpR6JCYl1zrzROQ,241,5.0
4,-hRpmcavsC0UDI_QsQsUcA,235,4.0


In [12]:
import pyspark

assert type(top_5_useful_positive) == pyspark.sql.dataframe.DataFrame, \
    "The top_useful_positive variable should be a Spark DataFrame."

submitted = AutograderHelper.parse_spark_dataframe(top_5_useful_positive)

In [13]:
# Autograder cell. This cell is worth 1 point (out of 20). This cell does not contain hidden tests.
# This cell deliberately includes answers to provide guidance on how this question is graded.

assert len(submitted) == 5, \
    "The result must have 5 rows."

top_useful_review_id = "1lGXlyq4MALOMx17vpBcoQ"
assert submitted["review_id"][0] == top_useful_review_id, \
    f'The first row should have review_id "{top_useful_review_id}" (this review has the most "useful" ratings)'

In [14]:
# Autograder cell. This cell is worth 4 points (out of 20). This cell contains hidden tests.

## -- CHECKINS --

Determine what hours of the day most checkins occur.

- Create a variable `hours_by_checkin_count`. This should be a PySpark DataFrame
- The DataFrame should be ordered by `count` and contain 24 rows
- The DataFrame should have these columns:
    - `hour` (the hour of the day as an integer, 0-23)
    - `count` (the number of checkins that occurred in that hour)


In [15]:
checkin.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- date: string (nullable = true)



In [16]:
checkin.first()

Row(business_id='--1UhMGODdWsrMastO9DZw', date='2016-04-26 19:49:16, 2016-08-30 18:36:57, 2016-10-15 02:45:18, 2016-11-18 01:54:50, 2017-04-20 18:39:06, 2017-05-03 17:58:02')

In [17]:
checkin.show()

+--------------------+--------------------+
|         business_id|                date|
+--------------------+--------------------+
|--1UhMGODdWsrMast...|2016-04-26 19:49:...|
|--6MefnULPED_I942...|2011-06-04 18:22:...|
|--7zmmkVg-IMGaXbu...|2014-12-29 19:25:...|
|--8LPVSo5i0Oo61X0...| 2016-07-08 16:43:30|
|--9QQLMTbFzLJ_oT-...|2010-06-26 17:39:...|
|--9e1ONYQuAa-CB_R...|2010-02-08 05:56:...|
|--DaPTJW3-tB1vP-P...|2012-06-03 17:46:...|
|--DdmeR16TRb3LsjG...|2012-11-02 21:26:...|
|--EF5N7P70J_UYBTP...|2018-05-25 19:52:...|
|--EX4rRznJrltyn-3...|2010-02-26 17:05:...|
|--FBCX-N37CMYDfs7...|2010-05-31 07:57:...|
|--FLdgM0GNpXVMn74...|2012-10-23 18:47:...|
|--GM_ORV2cYS-h38D...|2011-09-11 18:16:...|
|--Gc998IMjLn8yr-H...| 2014-07-01 01:20:47|
|--I7YYLada0tSLkOR...|2014-11-07 00:51:...|
|--KCl2FvVQpvjzmZS...|2011-07-29 16:53:...|
|--KQsXc-clkO7oHRq...|2010-05-02 23:57:...|
|--Rsj71PBe31h5Ylj...|2011-12-15 18:09:...|
|--S62v0QgkqQaVUhF...|2010-12-25 07:04:...|
|--SrzpvFLwP_YFwB_...|2011-02-10

In [18]:
%%time
import pyspark.sql.functions as f
from pyspark.sql.functions import udf, explode
from pyspark.sql.types import ArrayType,StringType
from pyspark.sql.functions import hour, col
from pyspark.sql.types import IntegerType

datesplit = udf(lambda x: x.split(','),ArrayType(StringType()))
datesplit1 = udf(lambda x: int(x.split()[1].split(':')[0]), IntegerType())

hours_by_checkin_count = checkin.withColumn("date_exploded", f.explode(datesplit(col("date")))) \
                      .withColumn('hour', datesplit1(col("date_exploded"))) \
                      .groupby(['hour']).count().sort('count', ascending=False)

hours_by_checkin_count.show(24) 

[Stage 9:>                                                          (0 + 1) / 1]

+----+-------+
|hour|  count|
+----+-------+
|   1|1561788|
|  19|1502271|
|   0|1491176|
|   2|1411255|
|  20|1350195|
|  23|1344117|
|  18|1272108|
|  22|1257437|
|  21|1238808|
|   3|1078939|
|  17|1006102|
|  16| 852076|
|   4| 747453|
|  15| 617830|
|   5| 485129|
|  14| 418340|
|   6| 321764|
|  13| 270145|
|   7| 231417|
|  12| 178910|
|   8| 151065|
|  11| 111769|
|   9| 100568|
|  10|  88486|
+----+-------+

CPU times: user 27.3 ms, sys: 9.69 ms, total: 36.9 ms
Wall time: 3min 12s


In [19]:
assert type(hours_by_checkin_count) == pyspark.sql.dataframe.DataFrame, \
    "The top_useful_positive variable should be a Spark DataFrame."

submitted = AutograderHelper.parse_spark_dataframe(hours_by_checkin_count)

In [20]:
# Autograder cell. This cell is worth 1 point (out of 20). This cell does not contain hidden tests.

assert len(submitted) == 24, \
    "The top_useful_positive DataFrame must have 24 rows."

assert submitted["hour"][0] == 1, \
    'The first row should have hour 1'

In [21]:
# Autograder cell. This cell is worth 4 points (out of 20). This cell contains hidden tests.

## -- COMMON USEFUL WORDS --

Write a function that takes a Spark DataFrame as a parameter and returns a Spark DataFrame of the 50 most common words from *useful* reviews and their counts.

- A "useful review" has 10 or more "useful" ratings.
- Convert the text to lower case.
- Use the provided `splitter()` function in a UDF to split the text into individual words.
- Exclude the words in the provided `STOP_WORDS` set.
- Returned DataFrame should have these columns:
    - `word`
    - `count`

In [22]:
import re
from pyspark.sql.functions import lower, col
import pyspark.sql.functions as funct


def splitter(text):
    WORD_RE = re.compile(r"[\w']+")
    return WORD_RE.findall(text)


    STOP_WORDS = {
    "a", "about", "above", "after", "again", "against", "aint", "all", "also", "although", "am", "an", "and", "any",
    "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "can",
    "check", "checked", "could", "did", "do", "does", "doing", "don", "down", "during", "each", "few", "for", "from",
    "further", "get", "go", "got", "had", "has", "have", "having", "he", "her", "here", "hers", "herself", "him",
    "himself", "his", "how", "however", "i", "i'd", "if", "i'm", "in", "into", "is", "it", "its", "it's", "itself",
    "i've", "just", "me", "more", "most", "my", "myself", "no", "nor", "not", "now", "of", "off", "on", "once", "one",
    "online", "only", "or", "other", "our", "ours", "ourselves", "out", "over", "own", "paid", "place", "s", "said", 
    "same", "service", "she", "should", "so", "some", "such", "t", "than", "that", "the", "their", "theirs", "them",
    "themselves", "then", "there", "these", "they", "this", "those", "through", "to", "too", "under", "until", "up",
    "us", "very", "was", "we", "went", "were", "we've", "what", "when", "where", "which", "while", "who", "whom",
    "why", "will", "with", "would", "you", "your", "yours", "yourself", "yourselves",
}

def common_useful_words(reviews, limit=50):
    
    STOP_WORDS = {
    "a", "about", "above", "after", "again", "against", "aint", "all", "also", "although", "am", "an", "and", "any",
    "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "can",
    "check", "checked", "could", "did", "do", "does", "doing", "don", "down", "during", "each", "few", "for", "from",
    "further", "get", "go", "got", "had", "has", "have", "having", "he", "her", "here", "hers", "herself", "him",
    "himself", "his", "how", "however", "i", "i'd", "if", "i'm", "in", "into", "is", "it", "its", "it's", "itself",
    "i've", "just", "me", "more", "most", "my", "myself", "no", "nor", "not", "now", "of", "off", "on", "once", "one",
    "online", "only", "or", "other", "our", "ours", "ourselves", "out", "over", "own", "paid", "place", "s", "said", 
    "same", "service", "she", "should", "so", "some", "such", "t", "than", "that", "the", "their", "theirs", "them",
    "themselves", "then", "there", "these", "they", "this", "those", "through", "to", "too", "under", "until", "up",
    "us", "very", "was", "we", "went", "were", "we've", "what", "when", "where", "which", "while", "who", "whom",
    "why", "will", "with", "would", "you", "your", "yours", "yourself", "yourselves",
}
    
    useful = review.filter(review["useful"]>=10)
    useful = useful.withColumn('text', lower(col('text')))

    split = udf(lambda x: splitter(x), ArrayType(StringType()))
    word = useful.select('text', split('text').alias('word'))
            
    data = word.withColumn('word', funct.explode('word'))
    
    count = data.groupBy('word').count().sort('count', ascending=False)
    
    most_common = count.filter(~funct.col('word').isin(STOP_WORDS)).limit(limit)
    
    return most_common

Now we'll run it on the `review` DataFrame

In [23]:
common_useful_words_counts = common_useful_words(review)

In [24]:
assert type(common_useful_words_counts) == pyspark.sql.dataframe.DataFrame, \
    "The common_useful_words_counts variable should be a Spark DataFrame."

submitted = AutograderHelper.parse_spark_dataframe(common_useful_words_counts)

In [25]:
# Autograder cell. This cell is worth 2 points (out of 20). This cell does not contain hidden tests.

assert len(submitted) == 50, \
    "The common_useful_words_counts DataFrame must have 24 rows."

assert submitted["word"][0] == 'like', \
    'The first row should have word "like"'

assert submitted["count"][0] == 101251, \
    'The first row should have count 101251'

In [26]:
# Autograder cell. This cell is worth 6 points (out of 20). This cell contains hidden tests.